# Text Preprocessing and Vectorization

1. Text Preprocessing code
2. BOW and TFIDF Representation
3. Working Session

In [1]:
from urllib.request import urlopen 
from urllib.parse import urlparse
import numpy as np
from urllib.error import HTTPError
from urllib.error import URLError

import nltk
from nltk.corpus import stopwords
import itertools
import spacy
import random
import datetime
import ssl

from spellchecker import SpellChecker
import itertools

import spacy
import pandas as pd 
import gensim
import ast
import datasets

import os
import wordcloud
from collections import Counter
from nltk.util import ngrams 
from bs4 import BeautifulSoup , NavigableString
import re 
import datetime 
import random
import pickle
import string
import urllib.request
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

import string
import pickle
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

In [2]:
class extract_stopwords():
    ''' this class is used to extract a list of long stop words list from 
        https://www.ranks.nl/stopwords 
        Thanks to these ppl for providing this list'''
    def __init__(self):
        pass
    
    def extract_stopwords(self):
        # Extracting long stop words
        user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
        
        headers={'User-Agent':user_agent} 
        cont = ssl._create_unverified_context()

        url = r'https://www.ranks.nl/stopwords'
        request=urllib.request.Request(url, None, headers)
        response = urllib.request.urlopen(request, context=cont)

        data = response.read()
        sw_soup = BeautifulSoup(data)
        tbls = sw_soup.find_all('table')
        req_tbl = tbls[3]

        for i, item in enumerate(req_tbl.findAll('tr')):
            stp_wd = [item.findAll('td')[i].findAll(text=True) for i in range(len(item.findAll('td')))]

        stop_words = list(itertools.chain.from_iterable(stp_wd))
        stop_words = [x.lower() for x in stop_words]
        return stop_words
    
    
class Preprocess_text():
    '''Used to preprocess text.
       Just pass True/False as the parameter to the operation you want to perform on the text and 
       call the preprocess_text() function by passing the text as argument. 
    '''
    def __init__(self, special_char_rem= True, rem_urls = True, rem_digits = True, spell_correction = False,
                 l_case = False, stop_words_rem = True, stemming=False, lemmatization_ = False,  rem_html_tags = True):
        
        self.special_char_rem = special_char_rem
        self.rem_digits = rem_digits
        self.l_case = l_case
        self.stop_words_rem = stop_words_rem
        self.stemming = stemming
        self.lemmatization_ = lemmatization_
        self.spell_correction = spell_correction
        self.rem_urls = rem_urls
        self.rem_html_tags = rem_html_tags
        
        if (self.lemmatization_ and self.stemming):
            # priority given to lemmatization
            self.stemming = False
        
        if self.lemmatization_:
            self.stemming = False
            
        elif self.stemming:
            self.lemmatization_ = False
        
            
        if self.lemmatization_:
            self.nlp = spacy.load("en_core_web_sm")
        
        if self.stemming:
            self.ps = nltk.porter.PorterStemmer()
        
        if self.stop_words_rem:
            try:
                self.stop_words = pickle.load(open('./long_stop_words_list.pkl', 'rb'))
            except:
                try:
                    self.stop_words = extract_stopwords().extract_stopwords()
                    pickle.dump(self.stop_words, open('./long_stop_words_list.pkl', 'wb'))
                except:
                    self.stop_words = stopwords('english')

            

    
    def remove_urls(self, text):
        soup = BeautifulSoup(text, "html.parser")
        [s.extract() for s in soup(['iframe', 'script'])]
        stripped_text = soup.get_text()
        stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
        return stripped_text
    
    def remove_html_tags(self, text):
        text = re.sub(r'<.*?>', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        return text
    
    def simple_stemming(self, text):
        stemmer=self.ps
        text = ' '.join([stemmer.stem(word) for word in text.split()])
        return text
    
    def simple_lemma(self, text, pos_tags = ['NOUN', 'ADJ', 'VERB', 'ADV']):
        doc = self.nlp(text)
        lemm_txt = [token.lemma_ for token in doc if token.pos_ in pos_tags]
        lemm_txt = ' '.join(lemm_txt)
        return lemm_txt
    

    def lower_case(self, text):
        text = ' '.join([token.lower() for token in text.split()])
        return text

    def remove_special_characters(self, text, remove_digits= True):
        pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
        text = re.sub(pattern, ' ', text)
        text = re.sub('\s+', ' ', text)
        return text
    
    def spell_corrector(self, text):
        spell = SpellChecker()
        tokens = text.split()
        tokens = [token.strip() for token in tokens]
        corrected_tokens = [spell.correction(token) for token in tokens]
        text = ' '.join(corrected_tokens)
        return text

    def remove_stopwords(self, text, is_lower_case=False):
        stopwords = self.stop_words
        tokens = text.split()
    #     tokens = nltk.word_tokenize(text)

        tokens = [token.strip() for token in tokens]

        if is_lower_case:
            filtered_tokens = [token for token in tokens if token not in stopwords]
        else:
            filtered_tokens = [token for token in tokens if token.lower() not in stopwords]

        filtered_text = ' '.join(filtered_tokens)    
        return filtered_text
    
    def preprocess_text(self, text):  
        
        if self.rem_urls:
            text = self.remove_urls(text)

        if self.rem_html_tags:
            text = self.remove_html_tags(text)
            
        if self.special_char_rem:
            text = self.remove_special_characters(text, remove_digits=self.rem_digits)
            
            
        if self.spell_correction:
            text = self.spell_corrector(text)

        if self.l_case:
            text = self.lower_case(text)

        if self.stop_words_rem:
            text = self.remove_stopwords(text)

        if self.stemming:
            text = self.simple_stemming(text)
            
        if self.lemmatization_:
            text = self.simple_lemma(text)
            
        self.text = text

        return self.text

In [3]:
# Dataset Ref:
# https://huggingface.co/datasets/fabiochiu/medium-articles
data = datasets.load_dataset('fabiochiu/medium-articles', data_files= 'medium_articles.csv')

# converting the dataset_dictionary object to pandas dataframe
data = pd.DataFrame.from_dict(data['train'])

Using custom data configuration fabiochiu--medium-articles-96791ff68926910d
Reusing dataset csv (C:\Users\Sridhar Kamoji\.cache\huggingface\datasets\fabiochiu___csv\fabiochiu--medium-articles-96791ff68926910d\0.0.0\652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
data = data.sample(n = 1000, random_state=32)
data.reset_index(drop = True, inplace = True)

In [5]:
# initialization of Text Preprocessing Object
pp_text = Preprocess_text(special_char_rem=True,
    rem_urls=True,
    rem_digits=True,
    spell_correction=False,
    l_case=True,
    stop_words_rem=True,
    stemming=False,
    lemmatization_=True,
    rem_html_tags=True)

In [7]:
data['title'][500]
pp_text.preprocess_text(data['title'][500])

'*HEALTY COACH*'

'healty coach'

In [8]:
# preprocessing the title

data['pp_title'] = [pp_text.preprocess_text(txt) for txt in data['title']]

### Bag of Words and TFIDF Vectorization

In [33]:
cvec = CountVectorizer() # Bow
tvec = TfidfVectorizer() # TFIDF

In [34]:
cvec.fit(data['pp_title'][:5].tolist())
tvec.fit(data['pp_title'][:5].tolist())

CountVectorizer()

TfidfVectorizer()

In [35]:
cvec_data = cvec.transform(data['pp_title'][:5].tolist())
tvec_data = tvec.transform(data['pp_title'][:5].tolist())

In [36]:
cvec_data = pd.DataFrame(cvec_data.toarray(), columns= cvec.get_feature_names())
cvec_data['text'] = data['pp_title'][:5].tolist()
cvec_data

,case,deutsch,improve,management,pain,phone,risk,strong,struggle,write,text
0,1,0,0,0,0,1,0,0,0,1,case write phone
1,0,0,0,0,0,0,0,1,1,0,struggle strong
2,0,0,1,1,0,0,1,0,0,0,improve risk management
3,0,1,0,0,0,0,0,0,0,0,deutsch
4,0,0,0,0,1,0,0,0,0,0,pain


In [37]:
list(zip(tvec.get_feature_names(), tvec.idf_))

[('case', 2.09861228866811),
 ('deutsch', 2.09861228866811),
 ('improve', 2.09861228866811),
 ('management', 2.09861228866811),
 ('pain', 2.09861228866811),
 ('phone', 2.09861228866811),
 ('risk', 2.09861228866811),
 ('strong', 2.09861228866811),
 ('struggle', 2.09861228866811),
 ('write', 2.09861228866811)]

In [38]:
tvec_data = pd.DataFrame(tvec_data.toarray(), columns= tvec.get_feature_names())
tvec_data['text'] = data['pp_title'][:5].tolist()
tvec_data

,case,deutsch,improve,management,pain,phone,risk,strong,struggle,write,text
0,0.57735,0.0,0.00000,0.00000,0.0,0.57735,0.00000,0.000000,0.000000,0.57735,case write phone
1,0.00000,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.707107,0.707107,0.00000,struggle strong
2,0.00000,0.0,0.57735,0.57735,0.0,0.00000,0.57735,0.000000,0.000000,0.00000,improve risk management
3,0.00000,1.0,0.00000,0.00000,0.0,0.00000,0.00000,0.000000,0.000000,0.00000,deutsch
4,0.00000,0.0,0.00000,0.00000,1.0,0.00000,0.00000,0.000000,0.000000,0.00000,pain


### Working Session (Audience)

1. Sentence Tokenize the text column
2. Preprocess the text column
3. Represent the data using BOW and Tfidf
4. Experiment with ngram_range, min_df, max_df etc